In diesem Notebook wird der Datensatz base_fake_real.csv verwendet und untersucht wie man Fake-Reviews von Echten Reviews unterscheiden kann. Zuerst wird der Datensatz so vorbeietet um Features zu generien. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import urllib
from fastai.vision.all import *
from fastdownload import download_url

Datensatz für Feature Extraction bzw. Feature Engineering vorbereitet


In [150]:
df = pd.read_csv("base_fake_real.csv")
print(f"Base Datensatz: {df.shape}")
# print(df.columns)

df.drop(
    [
        "index_fake",
        "org_text",
        "org_stars",
        "sent_score_0",
        "sent_v2",
        "sent_v3",
        "sent_v3.1",
        "prompt_v3",
        "website",
        "dalle_prompt",
        "website",
        "prompt_v2",
        "gpt3_v2",
        "gpt3_v3",
        "gpt3_v3.1",
        "prompt_v4",
        "org_reviewId",
        "sent_v4",
        "keywords",
        "keywords_only",
        "text_length",
        "reviewImageUrls/3",
        "reviewerPhotoUrl",
        "reviewerUrl",
        "reviewerId",
        "temporarilyClosed",
        "reviewsCount",
        "street",
        "state",
        "totalScore",
        "subTitle",
        "description",
        "price",
        "sentiment",
    ],
    axis=1,
    inplace=True,
)

df = df.reindex(
    columns=[
        "label",
        "reviewId",
        "placeId",
        "reviewUrl",
        "url",
        "title",
        "categoryName",
        "genre",
        "text",
        "stars",
        "publishedAtDate",
        "likesCount",
        "name",
        "isLocalGuide",
        "reviewerNumberOfReviews",
        "reviewImageUrls/0",
        "reviewImageUrls/1",
        "reviewImageUrls/2",
    ]
)

print(f"Reduzierter Datensatz als neue Basis für FE: {df.shape}")
print(df.columns)

df.to_csv("base_features.csv", index=False)
df.to_excel("base_features.xlsx", index=False)


Base Datensatz: (1410, 51)
Reduzierter Datensatz als neue Basis für FE: (1410, 18)
Index(['label', 'reviewId', 'placeId', 'reviewUrl', 'url', 'title',
       'categoryName', 'genre', 'text', 'stars', 'publishedAtDate',
       'likesCount', 'name', 'isLocalGuide', 'reviewerNumberOfReviews',
       'reviewImageUrls/0', 'reviewImageUrls/1', 'reviewImageUrls/2'],
      dtype='object')


## Bild:
1. Feature Extrahieren über Pretrained ResNet-18 Architektur und in Dataframe abspeichern. 

## Text:

1. Feature extrahieren über PNLP Paket, wie z.B: lenth, adverb, Großklein, Rechtschriebung, Smiley-Nutzung usw... wie bei Peter

## Tabulare Daten 

1. Zeitfeature umrechnen in kategorische Variablen

## Exkurs: Eigenen Img-Klassifikator trainiert und ausgwertet

Bilder für Training vorberieten und

In [ ]:
df = pd.read_csv('train_weitere_forschung_ohne_base.csv')

df = df.sample(n=500, random_state=9)
# df = df.sample(n=1000, random_state=42)
df = df.sort_index()

for index, row in df.iterrows():
    if index < 0:
        print(f"Index {index} schon heruntergeladen, skipped ..")
        continue
    for i in range(2):
        if not pd.isna(row[f'reviewImageUrls/{i}']):
            print(f"Downloading image for {index}...")
            url = row[f'reviewImageUrls/{i}']
            filename = f"{url.split('/')[-1]}.png"
            path = os.path.join('02_Images', 'train', 'real', filename)
            if not os.path.exists(path):
                try:
                    urllib.request.urlretrieve(url, path)
                except:
                    print(f'Error with {url}')

print("Done with downloading real images for training set.")

Start Umsetzung mit FASTAI visual Learning als Classifizierung Problem (AUF COLAB!)

In [ ]:
df = pd.read_csv('base_keywords_sentiment_reduced.csv')
print(df.shape)

df_all = pd.read_csv('01_Data/raw_data/dataset_weitere_forschung.csv')
print(df_all.shape)

df_all = df_all[['text', 'reviewId', 'url', 'placeId', 'categoryName', 'stars', 'title','reviewImageUrls/0','reviewImageUrls/1','reviewImageUrls/2','reviewImageUrls/3','reviewImageUrls/4','reviewImageUrls/5','genre']]
df_all = df_all[df_all['reviewImageUrls/0'].notna()]
print(df_all.shape)
df_all = df_all[~df_all['reviewId'].isin(df['reviewId'])]
print(df_all.shape)
print(df_all.columns)

image_urls_cols = [
    "reviewImageUrls/0",
    "reviewImageUrls/1",
    "reviewImageUrls/2",
    "reviewImageUrls/3",
    "reviewImageUrls/4",
    "reviewImageUrls/5",
]
for col in image_urls_cols:
    print(f"Processing col {col}")
    df_all[col] = df_all[col].str.replace("=w150-h150-k-no-p", "=w256-h256-p-k-no")

print(df_all.shape)

# df_all.to_csv('train_weitere_forschung_ohne_base.csv', index=False)
# df_all.to_excel('train_weitere_forschung_ohne_base.xlsx', index=False)

Hier nur die Auswertung. Training in colab.ipynb Notebook.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from matplotlib import colors

import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['font.family'] = 'Book Antiqua'

df = pd.read_csv('base_fake_real_imgcls_pred.csv')

y_true = df['label']
y_pred = df['prediction_0']

accuracy = accuracy_score(y_true, y_pred)
recall = classification_report(y_true, y_pred, output_dict=True)['weighted avg']['recall']
precision = classification_report(y_true, y_pred, output_dict=True)['weighted avg']['precision']
f1_score = classification_report(y_true, y_pred, output_dict=True)['weighted avg']['f1-score']

print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')
print(f'F1-Score: {f1_score:.4f}')

cmap = colors.ListedColormap(['#FFE5E4', '#D7F3D9'])

labels = y_true.unique()
fig, ax = plt.subplots(figsize=(5, 2.5))

cm = confusion_matrix(y_true, y_pred, labels=labels)


# Plot confusion matrix with custom colors
sns.heatmap(cm, annot=True, cmap=cmap, fmt='g', xticklabels=labels, yticklabels=labels, cbar=False)
plt.xlabel('Vorhersage')
plt.ylabel('Tatsächlich')

plt.savefig('confusion_matrix_green_red.svg', format='svg', bbox_inches='tight')